# NVIDIA-Pipecat Text to Speech Basics

The RivaTTSService class lets you use NVIDIA’s Riva Text-to-Speech (TTS) models, either on a local server or via NVIDIA’s cloud (NVCF). Below showcases a simple example using the NVCF endpoint to send text in, and audio out.

## Setup and Prerequisites
Before running this notebook, make sure you have:
- An NVIDIA API key for accessing cloud-hosted models via NVCF: [build.nvidia.com](build.nvidia.com)

## Setup Environment and Import Libraries

In [5]:
#Enter Your NVIDIA API KEY
import getpass
import os

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [6]:
import asyncio
import nest_asyncio
import os
import sys

from dotenv import load_dotenv
from pipecat.frames.frames import EndFrame, TTSSpeakFrame
from pipecat.pipeline.pipeline import Pipeline
from pipecat.pipeline.runner import PipelineRunner
from pipecat.pipeline.task import PipelineTask
from nvidia_pipecat.services.nvidia_llm import NvidiaLLMService
from nvidia_pipecat.services.riva_speech import RivaASRService, RivaTTSService
from pipecat.transports.local.audio import LocalAudioTransport, LocalAudioTransportParams

# Define the TTS Service
The RivaTTSService provides high-quality speech synthesis using NVIDIA’s Riva TTS models. It supports multiple voices, languages, and custom dictionaries for pronunciation. The RivaTTSService class specifically handles:  
- Connection to a TTS Server (API key or local)
- Voice selection
- Language and sample rate
- Model selection

In [7]:
# Define our text to speech service
tts = RivaTTSService(
    api_key=os.getenv("NVIDIA_API_KEY"), # set API Key
    voice_id= "English-US.Female-1",  # define the voice
    )

Enter any text below to generate speech and hear the result. You can try different messages to test how the voice sounds.

In [8]:
# Change the message to change the text to speech output, then run the next cell again
message="Hello there, how is it going!"

In [ ]:
async def main():
    transport = LocalAudioTransport(LocalAudioTransportParams(audio_out_enabled=True))

    pipeline = Pipeline([tts, transport.output()]) # We define our RivaTTS Service in the Pipeline

    task = PipelineTask(pipeline)

    async def say_something():
        await asyncio.sleep(1)
        await task.queue_frames([TTSSpeakFrame(message), EndFrame()])

    runner = PipelineRunner(handle_sigint=False if sys.platform == "win32" else True)

    await asyncio.gather(runner.run(task), say_something())


if __name__ == "__main__":
    nest_asyncio.apply()
    await main()